In [ ]:
import logging
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from telegram import Update, ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, ConversationHandler, CallbackContext

# Подключение к Google Sheets
def connect_to_google_sheets():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
    client = gspread.authorize(creds)
    return client.open("Название вашей таблицы").sheet1  # Укажите название вашей Google Таблицы

sheet = connect_to_google_sheets()

# Логирование
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

# Этапы диалога
SUM, CATEGORY, OPERATION_TYPE = range(3)

# Команда /start
def start(update: Update, context: CallbackContext):
    update.message.reply_text("Привет! Введите сумму:")
    return SUM

# Ввод суммы
def get_sum(update: Update, context: CallbackContext):
    context.user_data["sum"] = update.message.text

    # Кнопки для выбора категории
    keyboard = [["Еда", "Транспорт"], ["Развлечения", "Другое"]]
    reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True, resize_keyboard=True)

    update.message.reply_text("Выберите категорию:", reply_markup=reply_markup)
    return CATEGORY

# Выбор категории
def get_category(update: Update, context: CallbackContext):
    context.user_data["category"] = update.message.text

    # Кнопки для выбора типа операции
    keyboard = [["Доход", "Расход"]]
    reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True, resize_keyboard=True)

    update.message.reply_text("Выберите тип операции:", reply_markup=reply_markup)
    return OPERATION_TYPE

# Выбор типа операции и запись в Google Таблицу
def get_operation_type(update: Update, context: CallbackContext):
    context.user_data["operation_type"] = update.message.text

    # Запись данных в Google Sheets
    sheet.append_row([
        context.user_data["sum"],
        context.user_data["category"],
        context.user_data["operation_type"],
        update.message.date.strftime("%Y-%m-%d %H:%M:%S")
    ])

    update.message.reply_text("✅ Данные записаны! Введите /start, чтобы добавить еще.", reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

# Команда /cancel
def cancel(update: Update, context: CallbackContext):
    update.message.reply_text("Операция отменена. Для нового ввода введите /start.", reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

# Запуск бота
def main():
    updater = Updater("ВАШ_ТОКЕН_БОТА", use_context=True)
    dp = updater.dispatcher

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler("start", start)],
        states={
            SUM: [MessageHandler(Filters.text & ~Filters.command, get_sum)],
            CATEGORY: [MessageHandler(Filters.text & ~Filters.command, get_category)],
            OPERATION_TYPE: [MessageHandler(Filters.text & ~Filters.command, get_operation_type)],
        },
        fallbacks=[CommandHandler("cancel", cancel)],
    )

    dp.add_handler(conv_handler)

    updater.start_polling()
    updater.idle()

if name == "__main__":
    main()